In [14]:
import pandas as pd
import numpy as np

In [15]:
df = pd.read_csv('/Users/rahulshah/Desktop/Masking/submission (4).csv')

In [16]:
df.head(35)

,row_id,document,token,label_id,token_string,label
0,0,"Hi, I need a detailed report on how to improve...",0,0,hi,O
1,1,"Hi, I need a detailed report on how to improve...",1,0,",",O
2,2,"Hi, I need a detailed report on how to improve...",2,0,i,O
3,3,"Hi, I need a detailed report on how to improve...",3,0,need,O
4,4,"Hi, I need a detailed report on how to improve...",4,0,a,O
5,5,"Hi, I need a detailed report on how to improve...",5,0,detailed,O
6,6,"Hi, I need a detailed report on how to improve...",6,0,report,O
7,7,"Hi, I need a detailed report on how to improve...",7,0,on,O
8,8,"Hi, I need a detailed report on how to improve...",8,0,how,O
9,9,"Hi, I need a detailed report on how to improve...",9,0,to,O


In [20]:
df['sentence'] = df.groupby('document')['document'].transform('first')
def create_masked_sentence(group):
    masked_tokens = []
    for _, row in group.iterrows():
        if row['label'] != 'O':
            masked_tokens.append(f'[{row["label"]}]')
        else:
            masked_tokens.append(row['token_string'])
    return ' '.join(masked_tokens)

masked_sentence_dict = df.groupby('document').apply(create_masked_sentence).to_dict()
df['masked_sentence'] = df['document'].map(masked_sentence_dict)
df['unique_masked_sentence'] = df['masked_sentence'].where(~df.duplicated(subset=['masked_sentence'], keep='first'), '')
df.to_csv('/Users/rahulshah/Desktop/Masking/updated_submission.csv', index=False)


/var/folders/kc/2321vv0d3b1f61wy206xq6_40000gn/T/ipykernel_18310/3525299107.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  masked_sentence_dict = df.groupby('document').apply(create_masked_sentence).to_dict()


In [21]:
unique_df = df[['unique_sentence', 'unique_masked_sentence']].drop_duplicates().reset_index(drop=True)


In [23]:
print(unique_df.head(5))

                                     unique_sentence  \
0  Hi, I need a detailed report on how to improve...   
1                                                      
2  I need some advice on how to handle a PR crisi...   
3  5. Write an article about the impact of sleep ...   
4  Please create a pitch deck for Konopelski, Bin...   

                              unique_masked_sentence  
0  hi , i need a detailed report on how to improv...  
1                                                     
2  i need some advice on how to handle a pr crisi...  
3  5 . write an article about the impact of sleep...  
4  please create a pitch deck for [B-NAME] [I-NAM...  


In [25]:
unique_df.to_csv('/Users/rahulshah/Desktop/Masking/final_submission.csv', index=False)